# Vector DB used as Memory Demo

In [1]:
!pip install -r requirements.txt

  Using cached chromadb-0.3.26-py3-none-any.whl (123 kB)
  Using cached langchain-0.0.271-py3-none-any.whl (1.5 MB)
  Using cached hnswlib-0.7.0-cp310-cp310-win_amd64.whl
     -------------------------------------- 227.7/227.7 kB 1.7 MB/s eta 0:00:00
  Using cached duckdb-0.8.1-cp310-cp310-win_amd64.whl (9.8 MB)
     ---------------------------------------- 3.4/3.4 MB 8.1 MB/s eta 0:00:00
  Using cached onnxruntime-1.15.1-cp310-cp310-win_amd64.whl (6.7 MB)
  Using cached langsmith-0.0.33-py3-none-any.whl (36 kB)
     -------------------------------------- 511.3/511.3 kB 6.4 MB/s eta 0:00:00
  Using cached lz4-4.3.2-cp310-cp310-win_amd64.whl (99 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
     -------------------------------------- 145.4/145.4 kB 4.4 MB/s eta 0:00:00
  Using cached python_d

In [2]:
from modules.memory import Memory

In [3]:
memories = Memory(name="my_collection", db_directory="my_db_directory")

Collection 'my_collection' is ready.


# Add memories

In [7]:
# # Save some messages to the database
messages = ["Austin: My birthday is October 24th", "Austin: I work at a car auction as an IT guy", "Austin: My favorite food is steak"]
memories.save_messages_to_db(messages)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Message 'Austin: My birthday is October 24th' added to database.
Message 'Austin: I work at a car auction as an IT guy' is already in database. Skipped.
Message 'Austin: My favorite food is steak' is already in database. Skipped.


# Query the database

In [8]:
query_text = "Austin: Do you know my birthday?"
results = memories.query_db(query_text)
print(f"Query results for '{query_text}': {results}")

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Query results for 'Austin: Do you know my birthday?': {'ids': [['id1b5872ad-1df8-4aeb-a64e-3225237f937b', 'id77a1c9e1-5069-4325-8fd9-f54eadcd927a', 'idcacc6fa6-8108-4b27-8800-b33f7076bba4', 'idc6079b79-73df-444e-acfb-faa595464ce1']], 'embeddings': None, 'documents': [['Austin: My birthday is October 24th', 'Austin: My Birtday is October 24th', 'Austin: My favorite food is Steak', 'Austin: I work at a car Auction as an IT Guy']], 'metadatas': [[None, None, None, None]], 'distances': [[0.35002532601356506, 0.5655494928359985, 1.0889947414398193, 1.1685662269592285]]}


In [9]:
results_with_cutoff = memories.query_with_cutoff("Austin: When is my birthday?", 0.5)
results_with_cutoff

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


'Austin: My birthday is October 24th'

In [10]:
first_result = results['documents'][0][0]
print(f"Austin asked:\n{query_text} \nRemembered:\n{first_result}")


Austin asked:
Austin: Do you know my birthday? 
Remembered:
Austin: My birthday is October 24th


In [11]:
def query_with_cutoff(self, query_text, cutoff):
    results = self.query_db(query_text)
    if not results or not results['distances']:
        return "Nothing found."

    closest_distance = results['distances'][0][0]
    if closest_distance > cutoff:
        return "Nothing found."

    closest_result = results['documents'][0][0]
    return closest_result


In [12]:
# import wrapper
from langchain.llms import TextGen
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
from langchain.chains import (
    ConversationChain
)

In [13]:
# create instance with your endpoint
llm = TextGen(model_url="http://127.0.0.1:7861/")

In [14]:
llm("Question: What is the capital of France?\nAnswer:")

Question: What is the capital of France?
Answer: The capital city of France is Paris.


' The capital city of France is Paris.'

In [15]:
injected_memory = ""
template = f"""You are a chatbot having a conversation with a human.
{injected_memory}
{{history}}
Austin: {{input}}
Chatbot:"""

In [16]:
#function that will query the memory object db for the user's input and return the results if found. If not found, it will return an empty string.
def memory_find(query_text):
    results = memories.query_db(query_text)
    if results:
        result = f"[Memory - {results['documents'][0][0]}]\n"
        return result
    else:
        return ""
    

In [17]:
memory = ConversationBufferMemory(input_key="history", human_prefix="Austin")

In [18]:

prompt = PromptTemplate(
    input_variables=["history","input"], template=template
)

In [19]:
 # Create a conversation chain using the channel-specific memory
conversation = ConversationChain(
    prompt=prompt,
    llm=llm,
    verbose=True,
    memory=memory,
)

In [20]:
user_input = input("Enter Message: ")
injected_memory = memory_find("Austin:" + user_input)
print(injected_memory)

KeyboardInterrupt: Interrupted by user

In [21]:
template = f"""You are a chatbot having a conversation with a human.
{injected_memory}
{{history}}
Austin: {{input}}
Chatbot:"""
prompt = PromptTemplate(
    input_variables=["history","input"], template=template
)
template

'You are a chatbot having a conversation with a human.\n[Memory Austin: My Birtday is October 24th]\n\n{history}\nAustin: {input}\nChatbot:'

## take user input

In [1]:
conversation.predict(input=user_input, stop=["\nAustin:"])

NameError: name 'conversation' is not defined